In [16]:
import cv2
import sys
import os
#sys.path.append(os.path.abspath("./insightface/recognition/arcface_torch"))
#sys.path.append(os.path.abspath("./yolo5face"))
path_add = "./yolo5face"
sys.path.append(path_add)
from module_detect import YOLO5face_detector
for st in sys.path :
    if path_add in st :
        sys.path.remove(st)
        
path_add = "./insightface/recognition/arcface_torch"
sys.path.append(path_add)
from module_embedding import module_embedding
for st in sys.path :
    if path_add in st :
        sys.path.remove(st)
        
from IPython.display import Video
from pytictoc import TicToc
import numpy as np
import torch

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
t = TicToc()
path = "../sample/kbh3.webm"
root_pretrained = "../pretrained/"
device = "cuda:0"
#device = "cpu"
fourcc = cv2.VideoWriter_fourcc(*'DIVX')

In [3]:
vid = cv2.VideoCapture(path)

width  = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH) )  # float `width`
height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))  # float `height`
fps = vid.get(cv2.CAP_PROP_FPS)      # OpenCV2 version 2 used "CV_CAP_PROP_FPS"
frame_count = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))

duration = frame_count/fps
vid.release()

print("duration : " + str(duration))
print("fps : " + str(fps))
print("frame_count : " + str(frame_count))
print("width : " + str(width))
print("height : " + str(height))

duration : 11.332
fps : 1000.0
frame_count : 11332
width : 1280
height : 960


In [4]:
detector = YOLO5face_detector(root_pretrained + "yolov5n-0.5.pt", device )

Fusing layers... 


In [18]:
t.tic()
vid = cv2.VideoCapture(path)
out_yolo5face = cv2.VideoWriter('outputs/yolov5n05_crop.avi', fourcc, 20.0, (112,112))
while(vid.isOpened()):
    ret,frame = vid.read()
    if ret :
        #frame = cv2.resize(frame, dsize=(0, 0), fx=0.5, fy=0.5)
        xywh = detector.detect(frame)
        if xywh is not None : 
            crop = frame[xywh[1]:xywh[3],xywh[0]:xywh[2]]
            crop = cv2.resize(crop, dsize=(112,112))
            out_yolo5face.write(crop)
            #print(xywh)
        else :
            #print("None")
            pass
        
    else :
        break
t.toc()
vid.release()
out_yolo5face.release()

Elapsed time is 4.060175 seconds.


In [28]:
vid = cv2.VideoCapture('outputs/yolov5n05_crop.avi')
embedding = module_embedding("cuda:0",loss="arcface")

feat = None
t.tic()
while(vid.isOpened()):
    ret,frame = vid.read()
    if ret :
        frame = cv2.resize(frame, dsize=(112,112))
        t_feat = embedding.embed(frame)
        if feat is None :
            feat = t_feat
        else :
            feat = np.concatenate((feat,t_feat),0)
        #print(feat.shape)
    else :
        break
print(feat.shape)
feat = feat.mean(0)
print(feat.shape)
np.save("feat.npy",feat)
t.toc()
vid.release()

(160, 512)
(512,)
Elapsed time is 1.115493 seconds.


In [26]:
x = np.load("feat.npy")
x.shape

(512,)

In [29]:
y = "qwe3"
y[0:3]

'qwe'

In [31]:
np.random.rand(1)[0]

0.25973306694581955

In [32]:
d = {"A":1,"B":2,"C":3}

In [33]:
d

{'A': 1, 'B': 2, 'C': 3}

In [35]:
list(d.keys())

['A', 'B', 'C']

In [57]:
key = 'A'
t_key = 'A'
while t_key is key :
    t_key = list(d.keys())[np.random.randint(len(list(d.keys())))]
t_key

'B'